# Gemma NER

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(load_in_4bit=True)

# Use Gemma-2-9B
model_name = "google/gemma-2-9B-it"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    #quantization_config=bnb_config,
    device_map="cpu",
    torch_dtype=torch.bfloat16,
    token=os.getenv("HF_TOKEN"),
).bfloat16()
tokenizer = AutoTokenizer.from_pretrained(model_name)

/home/dvdblk/dev/swisshacks2024/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 16.82it/s]


In [3]:
# Define the input text
input_text = ""

# Encode the input text
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cpu")

In [4]:
# #model(input_ids=input_ids, return_dict=True, output_hidden_states=True)

# # Generate text
# output = model.generate(
#     input_ids,
#     max_length=10,
#     temperature=0.0,
#     #num_return_sequences=5,
#     pad_token_id=tokenizer.eos_token_id
# )

# print("Token ids:", output)

# # decode the output and print it
# print(tokenizer.decode(output[0], skip_special_tokens=True))

In [5]:
# load 'client_features.csv' from the data folder
with open(os.getenv("DATA_PATH") + "/client_features.csv", "r") as f:
    client_features_string = f.read()

In [33]:
import json

INSTRUCTIONS = """You are an AI assistant that analyzes transcripts and compares them to a database of people. ALWAYS provide your responses in the following JSON format:
{
"is_factually_correct": <yes|no>,
"reasoning": <Your short step-by-step reasoning here>
}

Ensure your response can be parsed as valid JSON (it has to start and end with curly braces and nothing else). Do not include any text outside of this JSON structure in your response."""


def check_transcript(transcript, model, tokenizer):
    chat = [
        {"role": "user", "content": f"""{INSTRUCTIONS}
        Here is a database of people:
        {client_features_string}

        And here is a transcript:
        "{transcript}"

        Is this transcript factually correct according to the database? Analyze the data and provide your response in the requested JSON format.
        """}
    ]
    chat2 = [
        {"role": "user", "content": "hi"}
    ]
    prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to("cpu")
    outputs = model.generate(**inputs, max_new_tokens=500)
    #response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)

    try:
        result = json.loads(response)
        return result
    except json.JSONDecodeError:
        return {"error": "Failed to parse JSON", "raw_output": response}

In [34]:
# load transcripts.csv file into a dictionary
import csv

transcripts = {}

with open(os.getenv("DATA_PATH") + "/transcriptions.csv", "r") as file:
    reader = csv.DictReader(file)
    for row in reader:
        transcripts[row["Audio File"]] = row["Transcription"]

In [35]:
tx = list(transcripts.values())[0]
print(check_transcript(tx, model, tokenizer))

{'is_factually_correct': 'no', 'reasoning': 'The provided Social Security Number 7667-587-79988 does not match any record in the database. Additionally, the name provided, Noya Chimurman, is not present in the database.'}
